In [ ]:
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import named_arrays as na
import msfc_ccd

In [ ]:
axis_x = "x"
axis_y = "y"
axis_xy = (axis_x, axis_y)

In [ ]:
dark = msfc_ccd.fits.open(
    path=msfc_ccd.samples.path_dark_esis1,
    axis_x=axis_x,
    axis_y=axis_y,
)

fig, ax = plt.subplots(
    figsize=(8, 4),
    constrained_layout=True,
)
im = na.plt.imshow(
    dark.outputs,
    axis_x=axis_x,
    axis_y=axis_y,
    ax=ax,
)
ax.set_xlabel("detector $x$ (pix)")
ax.set_ylabel("detector $y$ (pix)")
plt.colorbar(im.ndarray.item(), ax=ax, label="signal (DN)");

In [ ]:
axis_tap_x = "tap_x"
axis_tap_y = "tap_y"
axis_tap_xy = (axis_tap_x, axis_tap_y)

In [ ]:
taps = dark.taps(axis_tap_x, axis_tap_y)

In [ ]:
fig, ax = na.plt.subplots(
    axis_rows=axis_tap_y,
    axis_cols=axis_tap_x,
    nrows=taps.shape[axis_tap_y],
    ncols=taps.shape[axis_tap_x],
    sharex=True,
    constrained_layout=True,
)
na.plt.plot(
    taps.outputs.mean_trimmed(.01, axis_y),
    axis=axis_x,
    ax=ax,
)
na.plt.set_ylim(
    bottom=taps.outputs.percentile(5, axis=axis_xy),
    top=taps.outputs.percentile(95, axis=axis_xy),
    ax=ax,
)
na.plt.axvspan(
    xmin=0,
    xmax=taps.camera.sensor.num_blank,
    color="green",
    alpha=0.2,
    ax=ax,
    label="blank columns",
)
na.plt.axvspan(
    xmin=taps.num_x - taps.camera.sensor.num_overscan,
    xmax=taps.num_x,
    color="red",
    alpha=0.2,
    ax=ax,
    label="overscan columns",
)
na.plt.set_ylabel("row-averaged signal (DN)", ax[{axis_tap_x: 0}])
na.plt.set_xlabel("columns", ax=ax[{axis_tap_y: 0}])
na.plt.text(
    x=0.9,
    y=0.95,
    s=taps.label,
    ax=ax,
    transform=na.plt.transAxes(ax),
    ha="right",
    va="top",
)
ax.ndarray.flat[0].legend();

In [ ]:
bias_blank = taps.bias(num_blank=None, num_overscan=0)
bias_overscan = taps.bias(num_blank=0, num_overscan=None)

In [ ]:
unbiased_blank = taps - bias_blank
unbiased_overscan = taps - bias_overscan

In [ ]:
kwargs_filter = dict(
    size={axis_x: 11, axis_y: 11},
    proportion=0.05,
)
unbiased_blank = na.ndfilters.trimmed_mean_filter(unbiased_blank, **kwargs_filter)
unbiased_overscan = na.ndfilters.trimmed_mean_filter(unbiased_overscan, **kwargs_filter)

In [ ]:
dark_blank = dark.from_taps(unbiased_blank)
dark_overscan = dark.from_taps(unbiased_overscan)

In [ ]:
kwargs_hist = dict(
    axis=axis_xy,
    bins=na.arange(-2, 2, "xy", .1),
    density=True,
)
hist_blank = na.histogram(unbiased_blank.outputs, **kwargs_hist)
hist_overscan = na.histogram(unbiased_overscan.outputs, **kwargs_hist)

fig, ax = na.plt.subplots(
    axis_rows=axis_tap_y,
    axis_cols=axis_tap_x,
    nrows=taps.shape[axis_tap_y],
    ncols=taps.shape[axis_tap_x],
    sharex=True,
    sharey=True,
    constrained_layout=True,
)
na.plt.stairs(
    hist_blank.inputs,
    hist_blank.outputs,
    ax=ax,
    axis="xy",
    label="blank",
)
na.plt.stairs(
    hist_overscan.inputs,
    hist_overscan.outputs,
    ax=ax,
    axis="xy",
    label="overscan",
)
na.plt.text(
    x=0.95,
    y=0.95,
    s=taps.label,
    ax=ax,
    transform=na.plt.transAxes(ax),
    ha="right",
    va="top",
)
na.plt.axvline(0, ax=ax, color="black", linestyle="--")
na.plt.set_ylabel("probability density", ax[{axis_tap_x: 0}])
na.plt.set_xlabel("smoothed signal (DN)", ax=ax[{axis_tap_y: 0}])
ax[{axis_tap_x: 0, axis_tap_y: ~0}].ndarray.legend(loc="upper left");

In [ ]:
unbiased_blank.outputs.mean_trimmed(0.01, axis_xy)

In [ ]:
unbiased_overscan.outputs.mean_trimmed(0.01, axis_xy)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
colorizer = plt.Colorizer(
    norm=plt.Normalize(
        vmin=-1,
        vmax=1,
    ),
)
ani = na.plt.pcolormovie(
    na.ScalarArray(
        ndarray=np.array(["blank", "overscan"]),
        axes="blink",
    ),
    dark.inputs.pixel.x,
    dark.inputs.pixel.y,
    C=na.stack(
        arrays=[dark_blank.outputs, dark_overscan.outputs],
        axis="blink",
    ),
    axis_time="blink",
    ax=ax,
    kwargs_pcolormesh=dict(
        colorizer=colorizer,
    ),
    kwargs_animation=dict(
        interval=1000,
    )
)

ax.set_xlabel("detector $x$ (pix)")
ax.set_ylabel("detector $y$ (pix)")
plt.colorbar(
    mappable=plt.cm.ScalarMappable(colorizer=colorizer), 
    ax=ax,
    label="signal (DN)"
)
plt.close(fig)
IPython.display.HTML(ani.to_jshtml())